In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
data = pd.read_csv("/Users/yiweihan/Desktop/FeedForward_Data_hexa.csv")
X = data.values[:, 0:2] # Take only the first two features.
X = torch.tensor(X, dtype = torch.float)
y = data.values[:, 2]
y = torch.tensor(y, dtype = torch.long)

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 20)
        self.fc4 = nn.Linear(20, 20)
        self.fc5 = nn.Linear(20, 20)
        self.fc6 = nn.Linear(20, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        #x = F.relu(self.fc6(x))
        x = self.fc6(x)
        return F.log_softmax(x)
        #return F.softmax(x)

In [11]:
def plot_decision_boundary(clf, X, y, filename):
    # Set min and max values and give it some padding
    #x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    #y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    x_min, x_max = -1.0, 1.0
    y_min, y_max = -1.0, 1.0
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    #Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    X_out = net(torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype = torch.float))
    Z = X_out.data.max(1)[1]
    # Z.shape
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral, s = 1)
    plt.savefig(filename)
    plt.close()

In [8]:
net = Net()
learning_rate = .01
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9) 
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
nepochs = 10000 #10000
data, target = X, y
# run the main training loop
for epoch in range(nepochs):
    optimizer.zero_grad()
    net_out = net(data)
    loss = criterion(net_out, target)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch ', epoch, 'Loss ', loss.item())
        net_out = net(data)
        pred = net_out.data.max(1)[1] 
        correctidx = pred.eq(target.data)
        ncorrect = correctidx.sum()
        accuracy = ncorrect.item()/len(data)
        print('Training accuracy is ', accuracy)

<ipython-input-3-43b641510868>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch  0 Loss  0.801918625831604
Training accuracy is  0.05942453609796409
Epoch  100 Loss  0.0659443736076355
Training accuracy is  0.966869506423259
Epoch  200 Loss  0.05607996881008148
Training accuracy is  0.9751333483585005
Epoch  300 Loss  0.05665982514619827
Training accuracy is  0.9706257982120051
Epoch  400 Loss  0.051699455827474594
Training accuracy is  0.9752084741942754
Epoch  500 Loss  0.05419334024190903
Training accuracy is  0.9731800766283525
Epoch  600 Loss  0.05180956423282623
Training accuracy is  0.9756592292089249
Epoch  700 Loss  0.0484752394258976
Training accuracy is  0.9768612425813237
Epoch  800 Loss  0.048826757818460464
Training accuracy is  0.9764856134024491
Epoch  900 Loss  0.04698273539543152
Training accuracy is  0.9770114942528736
Epoch  1000 Loss  0.051329366862773895
Training accuracy is  0.9751333483585005
Epoch  1100 Loss  0.05001042038202286
Training accuracy is  0.9778378784463977
Epoch  1200 Loss  0.0441267155110836
Training accuracy is  0.9781

In [9]:
net_out = net(data)
pred = net_out.data.max(1)[1] # get the index of the max log-probability correctidx = pred.eq(target.data)
ncorrect = correctidx.sum()
accuracy = ncorrect.item()/len(data)
print('Training accuracy is ', accuracy)
print('Loss is ',loss)

Training accuracy is  0.9953421981819548
Loss is  tensor(0.0195, grad_fn=<NllLossBackward0>)


<ipython-input-3-43b641510868>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [12]:
plot_decision_boundary(net, X, y, 'Resultextra.pdf')

<ipython-input-3-43b641510868>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
#The training accuracy is 0.99534 and loss is 0.0195
#The number of hidden layer is 5
#The activation function is Relu
#The number of neurons per layer is 20